In [1]:
import pymongo
import pprint
import json
from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

In [8]:
with open('credential.json') as jfile:
    credential = json.load(jfile)

### Load Sample Data Through MongoDB Atlas

* Go to your Atlas account, click "Collections"
* Click "Load Sample Data"
* You can also see the data structure as well as some sample values through Atlas

In [7]:
# connect to the mongoclient
connection_str = credential['connection_str']
client = pymongo.MongoClient(connection_str)

client.list_database_names()

['sample_airbnb',
 'sample_analytics',
 'sample_geospatial',
 'sample_mflix',
 'sample_restaurants',
 'sample_supplies',
 'sample_training',
 'sample_weatherdata',
 'admin',
 'local']

In [9]:
db = client.sample_restaurants

db.list_collection_names()  # a collection is like a table

['restaurants', 'neighborhoods']

In [11]:
# count the number of documents in each collection
print(db['restaurants'].find().count())
print(db['neighborhoods'].find().count())

25359
195


In [12]:
db['restaurants'].find_one()

{'_id': ObjectId('5eb3d668b31de5d588f4292a'),
 'address': {'building': '2780',
  'coord': [-73.98241999999999, 40.579505],
  'street': 'Stillwell Avenue',
  'zipcode': '11224'},
 'borough': 'Brooklyn',
 'cuisine': 'American',
 'grades': [{'date': datetime.datetime(2014, 6, 10, 0, 0),
   'grade': 'A',
   'score': 5},
  {'date': datetime.datetime(2013, 6, 5, 0, 0), 'grade': 'A', 'score': 7},
  {'date': datetime.datetime(2012, 4, 13, 0, 0), 'grade': 'A', 'score': 12},
  {'date': datetime.datetime(2011, 10, 12, 0, 0), 'grade': 'A', 'score': 12}],
 'name': 'Riviera Caterer',
 'restaurant_id': '40356018'}

In [14]:
db['neighborhoods'].find_one().keys()

dict_keys(['_id', 'geometry', 'name'])

## MongoDB Index

### Default Index

* By default, mongoDB gives an id to each document in a collection, this is the default index of each collection.
* However, it's using "COLLSCAN" (collection scan), which means MongoDB needs to scan every document in a collection to look for the relevant documents.
* Default index can never be dropped

In [16]:
print(db.restaurants.index_information())
print(db.neighborhoods.index_information())

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'sample_restaurants.restaurants'}}
{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'sample_restaurants.neighborhoods'}}


In [17]:
pprint(db.restaurants.find().explain())

{'$clusterTime': {'clusterTime': Timestamp(1600551368, 1),
                  'signature': {'hash': b'R\xeey\xf5{\xf7\xf2\xec\xb2^\xfe\x9a'
                                        b'.\xf3\xb8\x8d\x81K\x81"',
                                'keyId': 6873329467658338307}},
 'executionStats': {'allPlansExecution': [],
                    'executionStages': {'advanced': 25359,
                                        'direction': 'forward',
                                        'docsExamined': 25359,
                                        'executionTimeMillisEstimate': 1,
                                        'isEOF': 1,
                                        'nReturned': 25359,
                                        'needTime': 1,
                                        'needYield': 0,
                                        'restoreState': 198,
                                        'saveState': 198,
                                        'stage': 'COLLSCAN',
                     

💡 If we check `executionStats` here, "stage" is "COLLSCAN" indicates it's using the collection scan. Therefore, `nReturned` and `totalDocsExamined` have the same amount of value, which is also the total number of documents, same as `db['restaurants'].find().count()` output.

### Type 1 Index - Single Field Index

* NOTE: cannot create the same index with different index names

In [19]:
# Without the index, it's COLLSCAN
pprint(db.restaurants.find({'cuisine':'Chinese'}).explain()['executionStats'])

{'allPlansExecution': [],
 'executionStages': {'advanced': 2418,
                     'direction': 'forward',
                     'docsExamined': 25359,
                     'executionTimeMillisEstimate': 1,
                     'filter': {'cuisine': {'$eq': 'Chinese'}},
                     'isEOF': 1,
                     'nReturned': 2418,
                     'needTime': 22942,
                     'needYield': 0,
                     'restoreState': 198,
                     'saveState': 198,
                     'stage': 'COLLSCAN',
                     'works': 25361},
 'executionSuccess': True,
 'executionTimeMillis': 10,
 'nReturned': 2418,
 'totalDocsExamined': 25359,
 'totalKeysExamined': 0}


In [20]:
# With the index
db.restaurants.create_index('cuisine', name='idx_cusine')

pprint(db.restaurants.find({'cuisine':'Chinese'}).explain()['executionStats'])

{'allPlansExecution': [],
 'executionStages': {'advanced': 2418,
                     'alreadyHasObj': 0,
                     'docsExamined': 2418,
                     'executionTimeMillisEstimate': 0,
                     'inputStage': {'advanced': 2418,
                                    'direction': 'forward',
                                    'dupsDropped': 0,
                                    'dupsTested': 0,
                                    'executionTimeMillisEstimate': 0,
                                    'indexBounds': {'cuisine': ['["Chinese", '
                                                                '"Chinese"]']},
                                    'indexName': 'idx_cusine',
                                    'indexVersion': 2,
                                    'isEOF': 1,
                                    'isMultiKey': False,
                                    'isPartial': False,
                                    'isSparse': False,
            

💡 
* Based on the comparison results here, after creating the index, the execution time and examined documents all decreased, "stage" has become "IXSCAN" which means scan with index
* In `create_index()` we can set `unique=True` if the index value has no duplicates, otherwise this setting will get duplicated index error

In [25]:
# here you can see index "idx_cusine" got created
db.restaurants.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'sample_restaurants.restaurants'},
 'idx_cusine': {'v': 2,
  'key': [('cuisine', 1)],
  'ns': 'sample_restaurants.restaurants'}}

In [26]:
# to drop the index
db.restaurants.drop_index('idx_cusine')

db.restaurants.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'sample_restaurants.restaurants'}}

### Type 2 Index - Compound Index

* it creates the index on multiple fields

In [32]:
db.restaurants.find_one({'cuisine':'Chinese'})

{'_id': ObjectId('5eb3d668b31de5d588f42936'),
 'address': {'building': '1269',
  'coord': [-73.871194, 40.6730975],
  'street': 'Sutter Avenue',
  'zipcode': '11208'},
 'borough': 'Brooklyn',
 'cuisine': 'Chinese',
 'grades': [{'date': datetime.datetime(2014, 9, 16, 0, 0),
   'grade': 'B',
   'score': 21},
  {'date': datetime.datetime(2013, 8, 28, 0, 0), 'grade': 'A', 'score': 7},
  {'date': datetime.datetime(2013, 4, 2, 0, 0), 'grade': 'C', 'score': 56},
  {'date': datetime.datetime(2012, 8, 15, 0, 0), 'grade': 'B', 'score': 27},
  {'date': datetime.datetime(2012, 3, 28, 0, 0), 'grade': 'B', 'score': 27}],
 'name': 'May May Kitchen',
 'restaurant_id': '40358429'}

In [33]:
# Without the index
pprint(db.restaurants.find({'borough':'Brooklyn','cuisine':'Chinese'}).explain()['executionStats'])

{'allPlansExecution': [],
 'executionStages': {'advanced': 763,
                     'direction': 'forward',
                     'docsExamined': 25359,
                     'executionTimeMillisEstimate': 0,
                     'filter': {'$and': [{'borough': {'$eq': 'Brooklyn'}},
                                         {'cuisine': {'$eq': 'Chinese'}}]},
                     'isEOF': 1,
                     'nReturned': 763,
                     'needTime': 24597,
                     'needYield': 0,
                     'restoreState': 198,
                     'saveState': 198,
                     'stage': 'COLLSCAN',
                     'works': 25361},
 'executionSuccess': True,
 'executionTimeMillis': 11,
 'nReturned': 763,
 'totalDocsExamined': 25359,
 'totalKeysExamined': 0}


In [34]:
# With the index
db.restaurants.create_index([('borough',pymongo.ASCENDING),
                            ('cuisine',pymongo.DESCENDING)],
                            name='idx_borough_cuisine')
# Get indexes
pprint(db.restaurants.index_information())

{'_id_': {'key': [('_id', 1)], 'ns': 'sample_restaurants.restaurants', 'v': 2},
 'idx_borough_cuisine': {'key': [('borough', 1), ('cuisine', -1)],
                         'ns': 'sample_restaurants.restaurants',
                         'v': 2}}


In [35]:
pprint(db.restaurants.find({'borough':'Brooklyn','cuisine':'Chinese'}).explain()['executionStats'])

{'allPlansExecution': [],
 'executionStages': {'advanced': 763,
                     'alreadyHasObj': 0,
                     'docsExamined': 763,
                     'executionTimeMillisEstimate': 0,
                     'inputStage': {'advanced': 763,
                                    'direction': 'forward',
                                    'dupsDropped': 0,
                                    'dupsTested': 0,
                                    'executionTimeMillisEstimate': 0,
                                    'indexBounds': {'borough': ['["Brooklyn", '
                                                                '"Brooklyn"]'],
                                                    'cuisine': ['["Chinese", '
                                                                '"Chinese"]']},
                                    'indexName': 'idx_borough_cuisine',
                                    'indexVersion': 2,
                                    'isEOF': 1,
              

### Type 3 Index - Multikey Index

* It creates index on fileds that're in array format

In [37]:
db.restaurants.find_one({'cuisine':'Chinese'})['grades']

[{'date': datetime.datetime(2014, 9, 16, 0, 0), 'grade': 'B', 'score': 21},
 {'date': datetime.datetime(2013, 8, 28, 0, 0), 'grade': 'A', 'score': 7},
 {'date': datetime.datetime(2013, 4, 2, 0, 0), 'grade': 'C', 'score': 56},
 {'date': datetime.datetime(2012, 8, 15, 0, 0), 'grade': 'B', 'score': 27},
 {'date': datetime.datetime(2012, 3, 28, 0, 0), 'grade': 'B', 'score': 27}]

In [42]:
db.restaurants.distinct('grades.grade')

['A', 'B', 'C', 'Not Yet Graded', 'P', 'Z']

In [43]:
pprint(db.restaurants.find({'grades.grade':'Z'}).explain()["executionStats"])

{'allPlansExecution': [],
 'executionStages': {'advanced': 1337,
                     'direction': 'forward',
                     'docsExamined': 25359,
                     'executionTimeMillisEstimate': 0,
                     'filter': {'grades.grade': {'$eq': 'Z'}},
                     'isEOF': 1,
                     'nReturned': 1337,
                     'needTime': 24023,
                     'needYield': 0,
                     'restoreState': 198,
                     'saveState': 198,
                     'stage': 'COLLSCAN',
                     'works': 25361},
 'executionSuccess': True,
 'executionTimeMillis': 26,
 'nReturned': 1337,
 'totalDocsExamined': 25359,
 'totalKeysExamined': 0}


In [44]:
db.restaurants.create_index('grades.grade', name='idx_grade')

# List indexes
pprint(db.restaurants.index_information())

{'_id_': {'key': [('_id', 1)], 'ns': 'sample_restaurants.restaurants', 'v': 2},
 'idx_borough_cuisine': {'key': [('borough', 1), ('cuisine', -1)],
                         'ns': 'sample_restaurants.restaurants',
                         'v': 2},
 'idx_grade': {'key': [('grades.grade', 1)],
               'ns': 'sample_restaurants.restaurants',
               'v': 2}}


In [45]:
pprint(db.restaurants.find({'grades.grade':'Z'}).explain()["executionStats"])

{'allPlansExecution': [],
 'executionStages': {'advanced': 1337,
                     'alreadyHasObj': 0,
                     'docsExamined': 1337,
                     'executionTimeMillisEstimate': 1,
                     'inputStage': {'advanced': 1337,
                                    'direction': 'forward',
                                    'dupsDropped': 0,
                                    'dupsTested': 1337,
                                    'executionTimeMillisEstimate': 0,
                                    'indexBounds': {'grades.grade': ['["Z", '
                                                                     '"Z"]']},
                                    'indexName': 'idx_grade',
                                    'indexVersion': 2,
                                    'isEOF': 1,
                                    'isMultiKey': True,
                                    'isPartial': False,
                                    'isSparse': False,
             

#### Adding partial index

* Index with filtering - only index part of the documents in a collection
* Reference: https://docs.mongodb.com/manual/core/index-partial/

In [79]:
db.restaurants.drop_indexes()

db.restaurants.create_index('grades.grade', name='idx_grade', partialFilterExpression={'grades.score':{'$gt':10}})
pprint(db.restaurants.index_information())

{'_id_': {'key': [('_id', 1)], 'ns': 'sample_restaurants.restaurants', 'v': 2},
 'idx_grade': {'key': [('grades.grade', 1)],
               'ns': 'sample_restaurants.restaurants',
               'partialFilterExpression': {'grades.score': SON([('$gt', 10)])},
               'v': 2}}


In [78]:
db.restaurants.find_one({'grades.grade':'Z', 'grades.score':{'$gt':10}})

{'_id': ObjectId('5eb3d668b31de5d588f4292f'),
 'address': {'building': '97-22',
  'coord': [-73.8601152, 40.7311739],
  'street': '63 Road',
  'zipcode': '11374'},
 'borough': 'Queens',
 'cuisine': 'Jewish/Kosher',
 'grades': [{'date': datetime.datetime(2014, 11, 24, 0, 0),
   'grade': 'Z',
   'score': 20},
  {'date': datetime.datetime(2013, 1, 17, 0, 0), 'grade': 'A', 'score': 13},
  {'date': datetime.datetime(2012, 8, 2, 0, 0), 'grade': 'A', 'score': 13},
  {'date': datetime.datetime(2011, 12, 15, 0, 0), 'grade': 'B', 'score': 25}],
 'name': 'Tov Kosher Kitchen',
 'restaurant_id': '40356068'}

In [80]:
# This cannot use the index
pprint(db.restaurants.find({'grades.grade':'Z', 'grades.score':{'$gt':5}}).explain()["executionStats"])

{'allPlansExecution': [],
 'executionStages': {'advanced': 1322,
                     'direction': 'forward',
                     'docsExamined': 25359,
                     'executionTimeMillisEstimate': 3,
                     'filter': {'$and': [{'grades.grade': {'$eq': 'Z'}},
                                         {'grades.score': {'$gt': 5}}]},
                     'isEOF': 1,
                     'nReturned': 1322,
                     'needTime': 24038,
                     'needYield': 0,
                     'restoreState': 198,
                     'saveState': 198,
                     'stage': 'COLLSCAN',
                     'works': 25361},
 'executionSuccess': True,
 'executionTimeMillis': 28,
 'nReturned': 1322,
 'totalDocsExamined': 25359,
 'totalKeysExamined': 0}


In [81]:
pprint(db.restaurants.find({'grades.grade':'Z', 'grades.score':{'$gt':10}}).explain()["executionStats"])

{'allPlansExecution': [],
 'executionStages': {'advanced': 1313,
                     'alreadyHasObj': 0,
                     'docsExamined': 1313,
                     'executionTimeMillisEstimate': 0,
                     'filter': {'grades.score': {'$gt': 10}},
                     'inputStage': {'advanced': 1313,
                                    'direction': 'forward',
                                    'dupsDropped': 0,
                                    'dupsTested': 1313,
                                    'executionTimeMillisEstimate': 0,
                                    'indexBounds': {'grades.grade': ['["Z", '
                                                                     '"Z"]']},
                                    'indexName': 'idx_grade',
                                    'indexVersion': 2,
                                    'isEOF': 1,
                                    'isMultiKey': True,
                                    'isPartial': True,
       

In [83]:
# Only when the filtering threshold is within the scope specified in `create_index()`, partial index would work
pprint(db.restaurants.find({'grades.grade':'Z', 'grades.score':{'$gt':20}}).explain()["executionStats"])

{'allPlansExecution': [],
 'executionStages': {'advanced': 1007,
                     'alreadyHasObj': 0,
                     'docsExamined': 1313,
                     'executionTimeMillisEstimate': 0,
                     'filter': {'grades.score': {'$gt': 20}},
                     'inputStage': {'advanced': 1313,
                                    'direction': 'forward',
                                    'dupsDropped': 0,
                                    'dupsTested': 1313,
                                    'executionTimeMillisEstimate': 0,
                                    'indexBounds': {'grades.grade': ['["Z", '
                                                                     '"Z"]']},
                                    'indexName': 'idx_grade',
                                    'indexVersion': 2,
                                    'isEOF': 1,
                                    'isMultiKey': True,
                                    'isPartial': True,
       

### Type 4 Index - Text Index

* It does search through text match, using text field
  * Doesn't need exact match, but more like substring match

In [48]:
# drop all the indexes first
db.restaurants.drop_indexes()

db.restaurants.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'sample_restaurants.restaurants'}}

In [50]:
db.restaurants.find_one()

{'_id': ObjectId('5eb3d668b31de5d588f4292a'),
 'address': {'building': '2780',
  'coord': [-73.98241999999999, 40.579505],
  'street': 'Stillwell Avenue',
  'zipcode': '11224'},
 'borough': 'Brooklyn',
 'cuisine': 'American',
 'grades': [{'date': datetime.datetime(2014, 6, 10, 0, 0),
   'grade': 'A',
   'score': 5},
  {'date': datetime.datetime(2013, 6, 5, 0, 0), 'grade': 'A', 'score': 7},
  {'date': datetime.datetime(2012, 4, 13, 0, 0), 'grade': 'A', 'score': 12},
  {'date': datetime.datetime(2011, 10, 12, 0, 0), 'grade': 'A', 'score': 12}],
 'name': 'Riviera Caterer',
 'restaurant_id': '40356018'}

In [55]:
db.restaurants.create_index([('name', 'text')], name='idx_restaurant_name')

pprint(db.restaurants.find_one({"$text": {"$search": "coffee bubble tea"}}))

{'_id': ObjectId('5eb3d669b31de5d588f48b5e'),
 'address': {'building': '65',
             'coord': [-73.999174, 40.7334443],
             'street': 'W 8Th St',
             'zipcode': '10011'},
 'borough': 'Manhattan',
 'cuisine': 'Chinese',
 'grades': [{'date': datetime.datetime(2015, 1, 20, 0, 0),
             'grade': 'Not Yet Graded',
             'score': 22}],
 'name': 'Bubble Tea',
 'restaurant_id': '50018503'}


### Type 5 Index - Geospatial Index

* It helps search nearby
  * `pymongo.GEO2D`
  * `pymongo.GEOSPHERE`
    * This one is very easy to create error, not suggestto use it. For more, check https://www.analyticsvidhya.com/blog/2020/09/mongodb-indexes-pymongo-tutorial/?utm_source=feedburner&utm_medium=email&utm_campaign=Feed%3A+AnalyticsVidhya+%28Analytics+Vidhya%29

In [56]:
db.restaurants.drop_indexes()

db.restaurants.find_one()

{'_id': ObjectId('5eb3d668b31de5d588f4292a'),
 'address': {'building': '2780',
  'coord': [-73.98241999999999, 40.579505],
  'street': 'Stillwell Avenue',
  'zipcode': '11224'},
 'borough': 'Brooklyn',
 'cuisine': 'American',
 'grades': [{'date': datetime.datetime(2014, 6, 10, 0, 0),
   'grade': 'A',
   'score': 5},
  {'date': datetime.datetime(2013, 6, 5, 0, 0), 'grade': 'A', 'score': 7},
  {'date': datetime.datetime(2012, 4, 13, 0, 0), 'grade': 'A', 'score': 12},
  {'date': datetime.datetime(2011, 10, 12, 0, 0), 'grade': 'A', 'score': 12}],
 'name': 'Riviera Caterer',
 'restaurant_id': '40356018'}

In [58]:
db.restaurants.create_index([('address.coord', pymongo.GEO2D)], name='idx_coord')

db.restaurants.find({'address.coord':{'$near':[-73,41]}}).count()

25357

In [60]:
pprint(db.restaurants.find_one({'address.coord':{'$near':[-73,41]}}))

{'_id': ObjectId('5eb3d668b31de5d588f42a40'),
 'address': {'building': '2701',
             'coord': [-72.7329266, 41.3099228],
             'street': 'Boston Post Road',
             'zipcode': '10467'},
 'borough': 'Bronx',
 'cuisine': 'Hamburgers',
 'grades': [{'date': datetime.datetime(2014, 5, 28, 0, 0),
             'grade': 'A',
             'score': 9},
            {'date': datetime.datetime(2014, 1, 16, 0, 0),
             'grade': 'A',
             'score': 5},
            {'date': datetime.datetime(2013, 1, 2, 0, 0),
             'grade': 'A',
             'score': 12},
            {'date': datetime.datetime(2011, 12, 27, 0, 0),
             'grade': 'A',
             'score': 2}],
 'name': 'White Castle',
 'restaurant_id': '40369669'}


In [61]:
# search nearby with specified min & max distance
db.restaurants.find({'address.coord':
                                    {'$near':[-73,41],
                                     '$minDistance':10,
                                     '$maxDistance':100
                                    }
                    }).count()

177